# Price Swing v1.1  by Tom Preston
## Capstone project for the Yves Hilpisch Algo Trading Certificate Program

This study is to analyze the price swings of the SP500 e-mini contract. Price swings move 
Final Project Proposal: “SP500 e-Mini Daytrading - Optimizing Pullback Entries” 

The SP500 e-Mini contract volatility has increased since Donald Trump became the U.S. President. The length of swings (measured in ticks) has increased and the number of swings per day has also increased. The question for the typical day trader is how/where do I enter a trade and what should my profit target be? A day trader needs some basic e-Mini swing descriptive statistics to plan and execute his day trading approach.

This proposed study will do 2 things:

1)	Develop descriptive statistics for SP500 e-mini 1 minute bar swings. (Note a swing high will be defined as high bar with 2 lower or equal bars before and after the swing high point and a similar approach for swing lows).  For example – how many ticks is a typical swing, how long does a swing go (in bars / time) before a pull back, does the swing have a breakout, if the swing has a breakout, how many bars does it go and how far past the previous swing high / low ? 

As shown in Figure 1 below, when the e-mini market seems to be moving in an “organized pattern, the market makes a series of higher highs and higher lows until the market reverses and makes a series of lower lows and lower highs. As a day trader, knowing that the down swings from 11:00 am to 12:00 pm EST typically average 20+ ticks in length down, 13 – 15 tick pull backs, and the breakouts below the previous swing lows only last for 1 – 2 bars is useful information. Figure 2 shows a similar structure.

Figure 3 is also a down trend however it’s more sideways trading. Seeing the swing lengths and number of bars in each swing (both with and against the trend) lets the trader develop an understanding of the current market environment and establish reasonable entry and stops.

2)	Optimize a pullback entry approach. Figures 1 – 3 all contain 3 exponential moving averages (ema). The blue line (21 ema) is a typical entry point and the red line (34 ema) and magenta (55 ema) show an approximate swing retracement area before the market continues in the larger trend direction.

My hypothesis is that a pull back entry at the 21 ema with a stop X ticks beyond the 55 ema can be profitable in the SP500 emini. I will need to leverage the existing backtesting code provided in our certificate program to develop pullback (i.e. limit entries) and then optimize these entries based upon the descriptive statistics analyses developed in step 1 above.

I will use the SP e-mini contract from Sept 2018 – Dec 2018 for my testing. 

Fig 1:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%201.png)

Fig 2:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%202.png)

Fig 3:
![SP500 1 min chart] (https://github.com/tom1presto/Algo-Trading-Certificate-Program/blob/master/SP500%20Study%20-%20Fig%203.png)

In [1]:
# Import Libraries
#
import pandas as pd
import numpy as np
import plotly.offline as pyo
import plotly.graph_objs as go
from plotly import __version__
print(__version__)

from datetime import datetime

# current date and time
now = datetime.now()
print(now)

3.7.0
2019-03-24 18:21:05.228772


In [2]:
# Reading ES 03 19 - Jan 1 - 18
# 
df = pd.read_csv(r"C:\Users\522147\Desktop\ES_03_19_2019_01_to_18_20secV2.csv", 
                    index_col=0, parse_dates=True, sep=",")
#  names=['date', 'open', 'high','low','open','volume'],|
df.head(10)

,open,high,low,close
Date_Time,,,,
2019-01-01 23:00:00,2493.00,2493.00,2492.75,2493.00
2019-01-01 23:00:20,2493.25,2493.50,2493.25,2493.50
2019-01-01 23:00:40,2493.25,2493.25,2493.00,2493.25
2019-01-01 23:01:00,2493.50,2493.50,2493.25,2493.25
2019-01-01 23:01:20,2493.25,2493.50,2493.25,2493.25
2019-01-01 23:01:40,2493.25,2493.25,2492.75,2492.75
2019-01-01 23:02:00,2492.75,2492.75,2492.75,2492.75
2019-01-01 23:02:20,2492.50,2492.50,2492.00,2492.00
2019-01-01 23:02:40,2492.00,2492.00,2491.75,2491.75


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 72361 entries, 2019-01-01 23:00:00 to 2019-01-18 17:00:00
Data columns (total 4 columns):
open     51953 non-null float64
high     51953 non-null float64
low      51953 non-null float64
close    51953 non-null float64
dtypes: float64(4)
memory usage: 2.8 MB


In [4]:
df.tail(10)

,open,high,low,close
Date_Time,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00


The 21 and 55 exponential moving average are key to this study. The 55 EMA will set the trend and pull backs to the 21 EMA will be used for entries

In [5]:
df['21ema'] = pd.Series.ewm(df['close'], span=21).mean()
df['55ema'] = pd.Series.ewm(df['close'], span=55).mean()

In [7]:
df.tail(10)

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259


## df_mini copy for testing sample

In [8]:
#
# Copying subset of data for testing
#
df_mini = df.iloc[-3000:].copy()

In [9]:
df_mini.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 3000 entries, 2019-01-18 00:20:20 to 2019-01-18 17:00:00
Data columns (total 6 columns):
open     2920 non-null float64
high     2920 non-null float64
low      2920 non-null float64
close    2920 non-null float64
21ema    3000 non-null float64
55ema    3000 non-null float64
dtypes: float64(6)
memory usage: 164.1 KB


In [10]:
df_mini.tail(10)

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259


In [11]:
df_mini.head()

,open,high,low,close,21ema,55ema
Date_Time,,,,,,
2019-01-18 00:20:20,2641.50,2641.50,2641.50,2641.50,2641.301647,2641.265153
2019-01-18 00:20:40,2641.25,2641.25,2641.00,2641.25,2641.296438,2641.264559
2019-01-18 00:21:00,NaN,NaN,NaN,NaN,2641.296438,2641.264559
2019-01-18 00:21:20,2641.50,2641.75,2641.50,2641.75,2641.345767,2641.284195
2019-01-18 00:21:40,2641.75,2641.75,2641.75,2641.75,2641.388959,2641.302948


This study will set the trend based upon the current bar being "x points" above or below the 55 EMA. A trend is long when the high is the current
bar is "trend points (set at 1.5)" above the 55EMA. The trend stays long until the low of a bar is "trend points" below the 55 EMA. The trend
then stays short until the long trend condition is set.

In [12]:
# Code works 2019-03-19 - moving above and dropping print statements
# 
# DF [0]open, [1]high, [2]low, [3]close, [4]21ema, [5]55ema   
#
trend_change_up_plot = {}    # dict with the trend change up price to plot 
trend_change_dn_plot = {}    # dict with the trend change down price to plot
trend_change_flag = {}       # either 1 (long) or -1 (short)
prev_trend = 1
current_trend = 1
trend_pts = 1.5              # trend points determines the trend

for i, row in df_mini.iterrows():
    current_bar = i
    if df_mini.index.get_loc(i) == 0:          # initial run through - set entries for first bar
        #print(f'Initial Row: {df_test.index.get_loc(i)} - Current Bar {current_bar} 21EMA: {row[4]:.2f} 55EMA {row[5]:.2f}')
        trend_change_up_plot[i] = row[3]   # set initial dict to have close value
        trend_change_flag[i] = 1     

        ##--------- Down Trend Change --------------------------------------------------------------------------------#    
    if current_trend == 1:                 # assume initial upswing - check 
        if row[2] <= row[5] - trend_pts:   # if low < 55EMA - trend points
            #print(f'  Down Trend: {df_test.index.get_loc(i)} {current_bar}: Low {row[2]:.2f} <= {row[5] - trend_pts:.2f} === 55EMA {row[5]:.2f} - {trend_pts}')
            prev_trend = current_trend
            trend_change_dn_plot[i] = row[2]   # set initial dict to have low
            trend_change_flag[i] =  -1   
            current_trend = -1
    
    ##--------- Up Trend Change --------------------------------------------------------------------------------#    
    if current_trend == -1:                # 
        if row[1] >= row[5] + trend_pts:   # if high > 55EMA + trend points
            #print(f'Up Trend: {df_test.index.get_loc(i)} {current_bar}: High {row[1]:.2f} >= {row[5] + trend_pts:.2f} === 55EMA {row[5]:.2f} + {trend_pts}')
            #print('')
            prev_trend = current_trend
            trend_change_up_plot[i] = row[1]
            trend_change_flag[i] =  1  
            current_trend = 1
            
#print(trend_change_up_plot)
#print(trend_change_dn_plot)
# print(trend_change_flag)

# print(trend_change_flag, file=open('testflag.txt', 'w'))

In [14]:
#
# Convert Trend Change up/down and trend change flags to dataframes. These data frames will be used to plot the trend changes on the price chart
#
df_trend_change_up_plot = pd.DataFrame.from_dict(trend_change_up_plot,orient='index',columns=['Price'])
df_trend_change_up_plot.index.names= ['Date_Time']  
#print(df_trend_change_up_plot)
#print(df_trend_change_up_plot.index)

df_trend_change_dn_plot = pd.DataFrame.from_dict(trend_change_dn_plot,orient='index',columns=['Price'])
df_trend_change_dn_plot.index.names= ['Date_Time']  
#print(df_trend_change_dn_plot)
#print(df_trend_change_dn_plot.index)

df_trend_change_flag = pd.DataFrame.from_dict(trend_change_flag,orient='index',columns=['trend'])
df_trend_change_flag.index.names= ['Date_Time']  
#print(df_trend_change_flag)
#print(df_trend_change_flag.index)

The trend flag dataframe is merged with the price chart data to establish price swings occuring with and against the trend

In [15]:
# 
# Merge trend change dataframe with the price data and reset the index
#
df_mini = pd.merge_ordered(df_mini, df_trend_change_flag, on='Date_Time',how='left',fill_method='ffill')
df_mini.set_index('Date_Time',inplace=True)

#print(df_mini.index)
#print(df_mini.head(20))

A new "up" price swing occurs when the current bar high exceeded the highest hi of the past 5 previous bars. A new "down" swing occurs
when the low of the current bar exceeds the low of the past 5 lowest bars.

The variables below hold the rolling 5 bar max highs and 5 bar min lows and the current bar crosses these values 

In [16]:
#
# set 5 previous bars max hi and min low and when the current bar crosses them
#

df_mini['trail_hi'] = df_mini['high'].rolling(5,min_periods=1).max()
df_mini['trail_hix'] = df_mini['high'] - df_mini['trail_hi'].shift(1)
df_mini['trail_lo'] = df_mini['low'].rolling(5,min_periods=1).min()
df_mini['trail_lox'] = df_mini['trail_lo'].shift(1) - df_mini['low']      
df_mini.tail(10)
#for i in range(200,400):
#    print(df_mini.iloc[i])

,open,high,low,close,21ema,55ema,trend,trail_hi,trail_hix,trail_lo,trail_lox
Date_Time,,,,,,,,,,,
2019-01-18 16:57:00,2676.25,2676.50,2676.25,2676.50,2675.190774,2674.365863,1,2677.00,-0.50,2675.50,-1.25
2019-01-18 16:57:20,2676.25,2676.50,2676.00,2676.25,2675.287105,2674.436017,1,2677.00,-0.50,2675.50,-0.50
2019-01-18 16:57:40,2676.00,2676.75,2676.00,2676.50,2675.397408,2674.512751,1,2676.75,-0.25,2676.00,-0.50
2019-01-18 16:58:00,2676.50,2676.75,2676.50,2676.75,2675.520412,2674.595805,1,2676.75,0.00,2676.00,-0.50
2019-01-18 16:58:20,2676.50,2677.25,2676.50,2676.75,2675.632226,2674.675664,1,2677.25,0.50,2676.00,-0.50
2019-01-18 16:58:40,2676.75,2677.25,2676.25,2676.75,2675.733869,2674.752457,1,2677.25,0.00,2676.00,-0.25
2019-01-18 16:59:00,2676.75,2677.50,2676.75,2677.25,2675.871733,2674.844798,1,2677.50,0.25,2676.00,-0.75
2019-01-18 16:59:20,2677.00,2677.00,2676.25,2676.25,2675.906129,2674.896686,1,2677.50,-0.50,2676.25,-0.25
2019-01-18 16:59:40,2676.25,2677.25,2676.25,2677.00,2676.005592,2674.974259,1,2677.50,-0.25,2676.25,0.00


The code below is the main logic for calculating swings and their length in price bars and points

The logic creates a separate file for up swings and down swings to be plotted on the chart separately

In [17]:
# Using iterrows to calc swings  -   latest version 2019-03-23
#
# 
# index 
# Date/Tim  [0]open, [1]high, [2]low, [3]close, [4]21ema, [5]55ema, [6]trend, [7]trail_hi, [8]trail_hix, [9]trail_lo, [10]trail_lox 
#
# initial var setup
up_down = 0
swing = 0
diff = 2.0
upswing_list = []
downswing_list = []
all_swing_list = []

debug_print = 0

print(datetime.now())
for i, row in df_mini.iterrows():
    if df_mini.index.get_loc(i) == 0:          # initial run through - set entries for first bar
        prev_swing_h = row[1]
        prev_swing_h_loc = i
        prev_swing_h_barnum = 0
        swing_h = row[1]
        swing_hl = row[2]
        swing_h_loc = i
        swing_h_barnum = 0
        swing_h_trend = 1
        prev_swing_l = row[2]
        prev_swing_l_loc = i
        prev_swing_l_barnum = 0 
        swing_l = row[2]
        swing_lh = row[1]
        swing_l_loc = i
        swing_l_barnum = 0
        swing_l_trend = 1
        if debug_print == 1:
            print(f'Row: {df_mini.index.get_loc(i)} Prv Swing High {prev_swing_h} PSHLoc {prev_swing_h_loc}')
            print(f'Row: {df_mini.index.get_loc(i)} Prv Swing Lo {prev_swing_l} PSHLoc {prev_swing_l_loc}')
    ##--------- UP SWING --------------------------------------------------------------------------------#    
    if swing == 1:                 # assume initial upswing - check 
        if row[1] > swing_h:       # if high > crrent hi, calc upswing stats,  make it >, not >= so the first peak is swing hi
            swing_h = row[1]
            swing_hl = row[2]
            swing_h_loc = i
            swing_h_barnum = df_mini.index.get_loc(i)
            up_pts = swing_h - prev_swing_l
            #print(f'Inside Up swing = 1 Row: {df_mini.index.get_loc(i)} Swing High {swing_h} PSHLoc {swing_h_loc} Up Pts: {up_pts}')
            #print(f'Inside Up swing = 1: {df_mini.index.get_loc(i)} Prv Swing Lo {prev_swing_l} PSHLoc {prev_swing_l_loc}')
                            
        if row[10] > 0:   # In Up swing and Down swing hit (lowest x lows crossed) 
            up_pts = swing_h - prev_swing_l
            up_swing_len = swing_h_barnum - prev_swing_l_barnum
            if debug_print == 1:
                print(f'$$ Calc UpSwing Time: Curr Swing Hi B#: {swing_h_barnum} Prev Swing L B#: {prev_swing_l_barnum}')            
                print(f'$$ Calc UpSwing Time: {swing_h_barnum - prev_swing_l_barnum}')
                print("")
                print(f' Low x - Up Swing ends - Row: {df_mini.index.get_loc(i)} Swing High {swing_h} PSHLoc {swing_h_loc} Up Pts: {up_pts}')
                print(f' Low x - Up Swing ends - Row: {df_mini.index.get_loc(i)} Prv Swing Lo {prev_swing_l} PSLLoc {prev_swing_l_loc}')
                print("")
            prev_swing_h = swing_h               # hold prev swing high after new formed down swing
            prev_swing_hl = swing_hl
            prev_swing_h_loc = swing_h_loc       #
            prev_swing_h_barnum = swing_h_barnum #  
            #up_swing_len = 99
            swing = -1                           # now down swing
            swing_l = row[2]                     # reset low to current low
            swing_l_loc = i
            swing_l_barnum = df_mini.index.get_loc(i)
            swing_l_trend = row[6]
            if debug_print == 1:
                print(f' {i} New Down Swing Trail_lox: {row[10]} New Swing Lo: {swing_l}')
                print(f' {i} End of Prev Up Swing: {swing_h} idx: {swing_h_loc}')                
                print('')
            #upswing_list.append(str(swing_h_loc))     # holding swing high - end of up swing
            upswing_list.append({'x0': str(swing_h_loc), 'high':prev_swing_h, 'low':prev_swing_hl})
            all_swing_list.append({'Date_Time':swing_h_loc, 'trend':swing_h_trend, 'swing':1, 'points':up_pts,'swing_len':up_swing_len})
    ##--------- DOWN SWING ------------------------------------------------------------------------------#    
    if swing == -1:                 # if current down swing 
        if row[2] < swing_l:        # new swing low - update current swing
            swing_l = row[2]
            swing_lh = row[1]
            swing_l_loc = i
            swing_l_barnum = df_mini.index.get_loc(i)
            dn_pts = prev_swing_h - swing_l
            if debug_print == 1:
                print(f'Inside Dn swing = 1 Row: {df_mini.index.get_loc(i)} Swing Lo {swing_l} PSLLoc {swing_l_loc} Dn Pts: {dn_pts}')
                print(f'Inside Dn swing = 1: {df_mini.index.get_loc(i)} Prv Swing Hi {prev_swing_h} PSHLoc {prev_swing_h_loc}')
            
            
        if row[8] > 0:   # In Down swing and Up swing hit (highest x highs crossed)
            dn_pts = prev_swing_h - swing_l
            dn_swing_len = swing_l_barnum - prev_swing_h_barnum
            if debug_print == 1:
                print(f'$$ Calc Down Swing Time: Curr Swing Lo B#: {swing_l_barnum} Prev Swing Hi B#: {prev_swing_h_barnum}')            
                print(f'$$ Calc Down Swing Time: {swing_l_barnum - prev_swing_h_barnum}')    
                print("")
                print(f'Hi x - Dn ends - Row: {df_mini.index.get_loc(i)} Swing Lo {swing_l} PSLLoc {swing_l_loc} Dn Pts: {dn_pts}')
                print(f'Hi x - Dn ends - Row: {df_mini.index.get_loc(i)} Prv Swing Hi {prev_swing_h} PSHLoc {prev_swing_h_loc}')
            
            prev_swing_l = swing_l               # hold prev swing low after new formed up swing
            prev_swing_lh = swing_lh
            prev_swing_l_loc = swing_l_loc       #
            prev_swing_l_barnum = swing_l_barnum
            swing = 1                            # now down swing
            swing_h = row[1]                     # reset high to current high
            swing_h_loc = i
            swing_h_barnum = df_mini.index.get_loc(i)
            swing_h_trend = row[6]
            if debug_print == 1:
                print(f' {i}     New Up Swing Trail_hix: {row[8]}')
                print(f' {i}     End of Prev Down Swing: {swing_l} idx: {swing_l_loc}') 
                print('')
            #downswing_list.append(str(swing_l_loc))     # holding swing low - end of down swing
            downswing_list.append({'x0': str(swing_l_loc), 'high':prev_swing_lh, 'low':prev_swing_l})
            all_swing_list.append({'Date_Time':swing_l_loc, 'trend':swing_l_trend,'swing':-1, 'points':dn_pts, 'swing_len':dn_swing_len})

    if df_mini.index.get_loc(i) == 0:     # set up after skipping over initial swing logic
        swing =1
        print(f'Set Swing: {swing}')
    

2019-03-24 18:22:43.681985
Set Swing: 1


In [18]:
#
# print df to text file for testing
#
print(type(all_swing_list))
print(len(all_swing_list))

with open("all_swing_list.txt", "w") as output:
    output.write(str(datetime.now())+"\n")
    for i in range(0,len(all_swing_list)):
        #print('in print loop')
        output.write(str(all_swing_list[i])+"\n")
    

<class 'list'>
286


In [19]:
print(len(upswing_list))
#print(upswing_list)

143


In [20]:
print(len(downswing_list))
#print(downswing_list)

143


In [21]:
# Converting DF index to string for plotly

df_mini_str = []
for i, row in df_mini.iterrows():
    df_mini_str.append(str(i))
    #print(test)

#print(df_mini_str)

In [22]:
# 
# Convert index to string for plotly. PLotly does not plot the time of a bar correctly using the date time stamp index but it does 
# plot correctly with the data time stamp converted to a string
#
df_trend_change_up_plot_str = []
df_trend_change_dn_plot_str = []
 
for i, row in df_trend_change_up_plot.iterrows():
    df_trend_change_up_plot_str.append(str(i))

for i, row in df_trend_change_dn_plot.iterrows():
    df_trend_change_dn_plot_str.append(str(i))

# print(df_trend_change_up_plot_str)
# print("")
#print(df_trend_change_dn_plot_str)

In [23]:
#
# creates line plot list for swing start - plotly needs a dictionary built for each plot
#
blue_dict = {'line':{'color': 'blue', 'width': 1}}
red_dict = {'line':{'color': 'red', 'width': 1}}

line_plot_list = []
zlist = []

def list_line_plot(list_name,color_plot):

    ctr = 0
    z1 = []
    z2 = []
    z3 = []
    z4 = []
    z5 = []
    z6 = []
    z7 = []
    z8 = []
    z9 = []
    z10 = []
    z11 = []

    #zlist = []
    #print(f'inside func - list_name: {list_name} len: {len(list_name)}')
#for item in item_list:
    for item in list_name:
        z1.append('type')
        z2.append('line')
        z3.append('x0')
        z4.append(item["x0"])      # bar date
        z5.append('y0')
        if item["low"] > 0:
            z6.append(item["low"] - 8)
            z10.append(item["low"] - 1)
        else:
            z6.append(2465.00)  
            z10.append(2470.00)  
        z7.append('x1')
        z8.append(item["x0"])      # bar date
        z9.append('y1')
        z11.append('line')

    for i1, i2, i3, i4, i5, i6, i7, i8, i9, i10 in zip(z1, z2, z3, z4, z5, z6, z7, z8, z9, z10):
        zlist.append({i1: i2, i3:i4, i5: i6, i7:i8, i9: i10})

    # print(f'zlist zip len: {len(zlist)} zlist: {zlist} ')    
    for item in zlist:
        item.update(color_plot)
    
    return zlist   # zlist is copied to line_list_plot
    #print(f'zlist zip len: {len(zlist)} zlist: {zlist} ')    
#---------------- End of List_Line_PLot function    

# creete downswing plots
list_line_plot(downswing_list,blue_dict)

# append down swing plots to the line_plot_list
line_plot_list = line_plot_list + zlist

# append upswing swing plots to the line_plot_list
zlist = []
list_line_plot(upswing_list,red_dict)
line_plot_list = line_plot_list + zlist

print_list = 0
if print_list == 1:
    with open("line_plot_list.txt", "w") as output:
        output.write(str(datetime.now())+"\n")
        for i in range(0,len(line_plot_list)):
            #print('in print loop')
            output.write(str(line_plot_list[i])+"\n")

This plotly charts the price data, the trend changes (green dot is a new up trend and a red dot is a new down trend), and the price swings.

The start of an up price swing is shown by a blue line and the start of a down swing is shown by a red line

The chart plots to html as this data is easier to view on a larger plot

In [26]:
# 
# PLotly with multiple plots 
#

trace1 = go.Ohlc(x=df_mini_str, 
                open=df_mini['open'],
                high=df_mini['high'],
                low=df_mini['low'],
                close=df_mini['close'],
                name='ES 20 Sec Bars',
                increasing=dict(line=dict(color= '#33CC33')),
                decreasing=dict(line=dict(color= '#CC3300')))

trace2 = go.Scatter(x=df_mini_str,
                            y=df_mini['21ema'],
                            mode='lines',name='21ema')
trace3 = go.Scatter(x=df_mini_str,
                            y=df_mini['55ema'],
                            mode='lines',name='55ema')

trace4 = go.Scatter(x=df_trend_change_up_plot_str,
                            y=df_trend_change_up_plot['Price'],
                            mode='markers',
                            marker = dict(
                            symbol = 'diamond',
                            #    symbol = 'triangle-up',
                            color = '#00ff00',
                            size = 10,
                            line = dict(width = 1)),
                            name='Up Trend')

trace5 = go.Scatter(x=df_trend_change_dn_plot_str,
                            y=df_trend_change_dn_plot['Price'],
                            mode='markers',
                            marker = dict(
                            symbol = 'diamond',
                            #    symbol = 'triangle-up',
                            color = '#ff0000',
                            size = 10,
                            line = dict(width = 1)),
                            name='Dn Trend')

layout = go.Layout(autosize=True,
                xaxis=go.layout.XAxis(rangeslider=dict(visible=False)),
                yaxis=go.layout.YAxis(automargin=True,autorange=True),
                shapes=line_plot_list)                 # zlist is updated below 
                  #shapes=zlist)                 # zlist is updated below

#print(trace1)
data = [trace1,trace2,trace3,trace4,trace5]

fig = go.Figure(data=data,layout=layout)
pyo.plot(fig, filename='styled_ohlc.html')

'file://C:\\Users\\522147\\Documents\\1 Python\\1 - Yves - Python for Finance\\Latest Files from TPQ\\styled_ohlc.html'

### End of Capstone Code